# Running VQE on an Amazon Braket backend

Let's review an example of running algorithms using Qiskit on Braket devices.

Qiskit provides a number of Algorithms and they are grouped by category according to the task they can perform. For instance Minimum Eigensolvers to find the smallest eigen value of an operator, for example ground state energy of a chemistry Hamiltonian or a solution to an optimization problem when expressed as an Ising Hamiltonian.

Algorithms are configurable and often part of the configuration will be in the form of smaller building blocks, of which different instances of the building block type can be given. For instance with `VQE`, the Variational Quantum Eigensolver, it takes a trial wavefunction, in the form of a `QuantumCircuit` and a classical optimizer among other things.

Let’s take a look at an example to construct a VQE instance. Here `TwoLocal` is the variational form (trial wavefunction), a parameterized circuit which can be varied, and SLSQP a classical optimizer. Then we pass the Hamiltonian to `compute_minimum_eigenvalue` method of `VQE` to get result.

Full example and explanation of algorithm detail you can find [here](https://qiskit-community.github.io/qiskit-algorithms/tutorials/01_algorithms_introduction.html).



In [1]:
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimatorV2
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.utils import algorithm_globals

from qiskit_braket_provider import BraketLocalBackend, BraketProvider

seed = 50
algorithm_globals.random_seed = seed

Get backend to run VQE with

In [2]:
provider = BraketProvider()
local_simulator = BraketLocalBackend()
local_simulator

BraketBackend[default]

## Running VQE



More docs on VQE and algorithms https://qiskit-community.github.io/qiskit-algorithms/tutorials/01_algorithms_introduction.html#A-complete-working-example

In [4]:
H2_op = SparsePauliOp(
    ["II", "IZ", "ZI", "ZZ", "XX"],
    coeffs=[
        -1.052373245772859,
        0.39793742484318045,
        -0.39793742484318045,
        -0.01128010425623538,
        0.18093119978423156,
    ],
)

estimator = BackendEstimatorV2(
    backend = local_simulator,
    options={"seed_simulator": seed},
)
ansatz = TwoLocal(2, rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1)

vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=slsqp)

result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 9,
    'eigenvalue': -1.3792822927523654,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x3143d76e0>,
    'optimal_parameters': {   ParameterVectorElement(θ[1]): -2.940960526404766,
                              ParameterVectorElement(θ[3]): -1.2903442440134034,
                              ParameterVectorElement(θ[5]): -4.710416180758196,
                              ParameterVectorElement(θ[2]): 6.115830155139623,
                              ParameterVectorElement(θ[4]): -2.1556830785333663,
                              ParameterVectorElement(θ[6]): -4.882565275402284,
                              ParameterVectorElement(θ[7]): 6.08932895996748,
                              ParameterVectorElement(θ[0]): -1.227012055676525},
    'optimal_point': array([-1.22701206, -2.94096053,  6.11583016, -1.29034424, -2.15568308,
       -4.71041618, -4.88256528,  6.08932896]),
    'optimal_va